In [1]:
!pip install pandas
!pip install requests


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import os
import requests
import time
import re

In [18]:
import re

def extract_year_period_week(filename):
    """Extracts year, period, and week from a filename like '2024_p1_w1.csv'.

    Args:
        filename: The name of the file.

    Returns:
        A tuple containing the year, period, and week as integers.
    """
    match = re.search(r"(\d+)_p(\d+)_w(\d+)", filename)
    if match:
        year = str(match.group(1))
        period = int(match.group(2))
        week = int(match.group(3))
        return year, period, week
    else:
        return None, None, None  # Handle cases where the pattern is not found

In [19]:
import pandas as pd

for filename in ['data/2024_p1_w1.csv']:
    df = pd.read_csv(filename, dtype={'GROWER': str, 'TERRAIN': str})
    year, period, week = extract_year_period_week(filename)

    if year is not None and period is not None and week is not None:
        df['year'] = year  # Add 'Year' column
        df['period'] = period  # Add 'Period' column
        df['week'] = week  # Add 'Week' column
    else:
        print(f"Warning: Could not extract year, period, and week from {filename}")

    # Process the DataFrame further...

In [20]:
df.head()

,GROWER,TERRAIN,WK_10,WK_11,WK_12,WK_13,WK_14,WK_15,WK_16,WK_17,...,STEMS,KLS.A,BXS.A,KLS.B,BXS.B,startdate,enddate,year,period,week
0,A. NOEL,0304,0,0,18,68,33,12,1,0,...,85,1180.87,88.12,59.57,4.45,25-03-2024,30-03-2024,2024,1,1
1,RABIA,0304,0,0,7,9,2,1,0,0,...,8,134.45,10.03,9.60,0.72,25-03-2024,30-03-2024,2024,1,1
2,"ALVAREZ, E.",0006,0,0,45,42,53,17,12,0,...,186,2746.25,204.95,58.18,4.34,25-03-2024,30-03-2024,2024,1,1
3,"ONGKINGCO, J.",0013,0,0,8,36,53,21,2,0,...,138,2288.52,170.78,21.40,1.60,25-03-2024,30-03-2024,2024,1,1
4,"REYES, P.",0027,0,0,6,0,82,20,1,0,...,88,1372.31,102.44,18.48,1.38,25-03-2024,30-03-2024,2024,1,1


In [21]:
melted_df = pd.melt(df,
                   id_vars=['GROWER',
                             'TERRAIN',
                             'BAGGING',
                             'STEMS',
                             'KLS.A',
                             'BXS.A',
                             'KLS.B',
                             'BXS.B',
                            'startdate','enddate','year','period','week'],
                   value_vars=['WK_10', 'WK_11', 'WK_12', 'WK_13',
                               'WK_14', 'WK_15', 'WK_16', 'WK_17'],
                   var_name='Week_Name',
                   value_name='stem_harvest_week_count')

display(melted_df.head())

,GROWER,TERRAIN,BAGGING,STEMS,KLS.A,BXS.A,KLS.B,BXS.B,startdate,enddate,year,period,week,Week_Name,stem_harvest_week_count
0,A. NOEL,0304,198,85,1180.87,88.12,59.57,4.45,25-03-2024,30-03-2024,2024,1,1,WK_10,0
1,RABIA,0304,73,8,134.45,10.03,9.60,0.72,25-03-2024,30-03-2024,2024,1,1,WK_10,0
2,"ALVAREZ, E.",0006,176,186,2746.25,204.95,58.18,4.34,25-03-2024,30-03-2024,2024,1,1,WK_10,0
3,"ONGKINGCO, J.",0013,129,138,2288.52,170.78,21.40,1.60,25-03-2024,30-03-2024,2024,1,1,WK_10,0
4,"REYES, P.",0027,96,88,1372.31,102.44,18.48,1.38,25-03-2024,30-03-2024,2024,1,1,WK_10,0


In [22]:
def rename_csv_files(directory):
  """Renames CSV files in a directory from 'p{period}_w{week}.csv' to 'processed_p{period}_w{week}.csv'.

  Args:
    directory: The directory containing the CSV files.
  """
  for filename in os.listdir(directory):
    if filename.endswith(".csv") and filename.startswith("p"):
      base_name = os.path.splitext(filename)[0]  # Get filename without extension
      period, week = base_name[1:].split("_")  # Extract period and week
      new_filename = f"processed_{period}_{week}.csv"
      os.rename(os.path.join(directory, filename), os.path.join(directory, new_filename))
      print(f"Renamed '{filename}' to '{new_filename}'")

In [ ]:
import requests
import time
import json
from datetime import datetime

# Function to convert date format
def format_date(date_str):
    try:
        # Convert from "dd-MM-yyyy" to "yyyy-MM-dd"
        return datetime.strptime(date_str, "%d-%m-%Y").strftime("%Y-%m-%d")
    except ValueError:
        print(f"Invalid date format: {date_str}")
        return None  # Handle invalid dates

# List of Strapi API endpoints
strapi_api_endpoints = [
    "http://127.0.0.1:1337/api/harvests"
]

# Iterate through the rows of the melted_df
for index, row in melted_df.iterrows():
    # Format the date fields
    formatted_startdate = format_date(str(row["startdate"]))
    formatted_enddate = format_date(str(row["enddate"]))

    # Ensure all values are correctly formatted
    data = {
        "data": {
            "grower": str(row["GROWER"]),
            "terrainid": str(row["TERRAIN"]),
            "bagging": int(row["BAGGING"]),
            "stems": int(row["STEMS"]),
            "class_type_a": float(row["KLS.A"]),
            "box_type_a": float(row["BXS.A"]),
            "class_type_b": float(row["KLS.B"]),
            "box_type_b": float(row["BXS.B"]),                        
            "stem_harvest_week": str(row["Week_Name"]),
            "stem_harvest_count": int(row["stem_harvest_week_count"]),
            "startdate": formatted_startdate,  # Fixed format
            "enddate": formatted_enddate,      # Fixed format
            "period": int(row["period"]),
            "week": int(row["week"]),
            "year": str(row["year"]),
        }
    }

    # Print formatted JSON to check before sending
    # print(f"Sending JSON for row {index}:\n{json.dumps(data, indent=2)}")

    for endpoint in strapi_api_endpoints:
        try:
            print(f'Posting to {endpoint} - ROW Index: {index}')
            response = requests.post(endpoint, json=data)
            response.raise_for_status()  # Raise an exception for bad status codes
            print(f"Success at {endpoint}: {response.json()}")

        except requests.exceptions.RequestException as e:
            print(f"Error posting to {endpoint}: {e}")

    time.sleep(0.3)


Sending JSON for row 0:
{
  "data": {
    "grower": "A. NOEL",
    "terrainid": "0304",
    "bagging": 198,
    "stems": 85,
    "class_type_a": 1180.87,
    "box_type_a": 88.12,
    "class_type_b": 59.57,
    "box_type_b": 4.45,
    "stem_harvest_week": "WK_10",
    "stem_harvest_count": 0,
    "startdate": "2024-03-25",
    "enddate": "2024-03-30",
    "period": 1,
    "week": 1,
    "year": "2024"
  }
}
Posting to http://127.0.0.1:1337/api/harvests - ROW Index: 0
Success at http://127.0.0.1:1337/api/harvests: {'data': {'id': 20, 'documentId': 'h9spviwu33pi3qimf5hp0sro', 'period': 1, 'week': 1, 'stem_harvest_count': 0, 'createdAt': '2025-03-27T18:04:06.681Z', 'updatedAt': '2025-03-27T18:04:06.681Z', 'publishedAt': '2025-03-27T18:04:06.679Z', 'terrainid': '0304', 'week_name': None, 'stem_harvest_week': 'WK_10', 'startdate': '2024-03-25', 'enddate': '2024-03-30', 'year': '2024', 'stems': 85, 'grower': 'A. NOEL', 'bagging': 198, 'box_type_a': 88.12, 'box_type_b': 4.45, 'class_type_a': 1

KeyboardInterrupt: 

In [ ]:
# prompt: from the melted_df save it to a csv
new_filename = f'processed_{file_name.split('_')[0][1]}_{file_name.split('_')[1].split('.')[0][1]}.csv'
melted_df.to_csv(new_filename, index=False)